In [1]:
import os,glob,sys
import numpy as np
import pandas as pd
import random
import pyslha
import time
import progressbar as P
import tempfile,gzip,pylhe
from atlas_topq_2019_23_Recast import getLHEevents,getPTThist
from atlas_topq_2019_23_Limits import read_ATLASdata,getSMLO
sys.path.append('/home/lessa/MonoXSMS')
from statisticalTools.simplifiedLikelihoods import Data,UpperLimitComputer

In [2]:
# ### Load ATLAS data and BG
xsecsObs,sm,covMatrix = read_ATLASdata()

# ### Load LO background from MG5
smLO = getSMLO()

# Get k-factor for each bin
kfac = np.divide(sm,smLO)

print(kfac)

[1.77263145 1.61467064 1.52121939 1.40926446 1.23749754 1.01519651
 0.90870208 0.58598112]


In [3]:
atlas_bins = np.array([355.0,381.0,420.0,478.0,549.0,633.0,720.0,836.0,2000.0])
bin_width  = atlas_bins[1:]-atlas_bins[:-1]


recastData = pd.read_pickle('./pp2tt_formFactors_scan.pcl')
binCols = [c for c in recastData.columns 
            if 'bin_' in c.lower() and not 'error' in c.lower()]
bins_left = np.array([eval(c.split('_')[1]) for c in binCols])
bins_right = np.array([eval(c.split('_')[2]) for c in binCols])

In [4]:
ulComp = UpperLimitComputer()
for ipt,pt in recastData.iterrows():
    signal = list(zip(bins_left,pt[binCols].values))
    signal = np.array(sorted(signal))[:,1]
    yDM = pt['yDM']
    # Make sure signal is normalized to yDM = 1
    signal = signal/yDM**2

    # Rescale predictions by bin-dependent k-factors and divide by the bin widths
    signal = kfac*signal/bin_width
    lumi = 139.0*1e3 # xsecs are in pb
    fac = bin_width*lumi
    data = Data(observed=xsecsObs*fac, backgrounds=sm*fac, 
                    covariance=covMatrix*fac**2, 
                    nsignal=signal*fac,deltas_rel=0.0)

    ul = ulComp.getUpperLimitOnMu(data)
    if not ul:
        print(ul)
        continue
    yDMmax = np.sqrt(ul)
    print(yDMmax,pt['yDM (95% C.L.)'])
    break

None
None
None
None
None
None
None
None
None
0.17385564868227743 1.0575901080611734


In [5]:
gg = pd.read_pickle('./gg2tt_formFactors_scan.pcl')
qq = pd.read_pickle('./qq2tt_formFactors_scan.pcl')

In [6]:
gg.iloc[0].T

model                                        Top-FormFactorsOneLoop-UFO
mST                                                               300.0
mChi                                                                5.0
mT                                                                172.5
yDM                                                                 1.0
process                                             $g g \to \bar{t} t$
xsec (pb)                                                     -0.001036
MC Events                                                         75000
file                  ../../processFolders/gg2tt_formFactors_scan/Ev...
bin_355_381                                                   -0.000176
bin_355_381_Error                                              0.000016
bin_381_420                                                    -0.00026
bin_381_420_Error                                              0.000016
bin_420_478                                                   -0

In [7]:
qq.iloc[0].T

model                                        Top-FormFactorsOneLoop-UFO
mST                                                               300.0
mChi                                                                5.0
mT                                                                172.5
yDM                                                                 1.0
process                                             $q q \to \bar{t} t$
xsec (pb)                                                     -0.000042
MC Events                                                         75000
file                  ../../processFolders/qq2tt_formFactors_scan/Ev...
bin_355_381                                                    0.000063
bin_355_381_Error                                              0.000004
bin_381_420                                                    0.000059
bin_381_420_Error                                              0.000004
bin_420_478                                                    0

In [8]:
len(gg),len(qq)

(207, 207)

In [9]:
gg['file'].value_counts()

../../processFolders/gg2tt_formFactors_scan/Events/run_01/unweighted_events.lhe.gz     1
../../processFolders/gg2tt_formFactors_scan/Events/run_127/unweighted_events.lhe.gz    1
../../processFolders/gg2tt_formFactors_scan/Events/run_139/unweighted_events.lhe.gz    1
../../processFolders/gg2tt_formFactors_scan/Events/run_292/unweighted_events.lhe.gz    1
../../processFolders/gg2tt_formFactors_scan/Events/run_151/unweighted_events.lhe.gz    1
                                                                                      ..
../../processFolders/gg2tt_formFactors_scan/Events/run_101/unweighted_events.lhe.gz    1
../../processFolders/gg2tt_formFactors_scan/Events/run_351/unweighted_events.lhe.gz    1
../../processFolders/gg2tt_formFactors_scan/Events/run_51/unweighted_events.lhe.gz     1
../../processFolders/gg2tt_formFactors_scan/Events/run_355/unweighted_events.lhe.gz    1
../../processFolders/gg2tt_formFactors_scan/Events/run_233/unweighted_events.lhe.gz    1
Name: file, Length: 2

In [10]:
pt = pd.read_pickle('../../processFolders/gg2tt_formFactors_scan/Events/run_01/unweighted_events_atlas_topq_2019_23.pcl')


In [11]:
for f in glob.glob('./qq2tt_formFactors_scan/Events/run_*/unweighted_events_atlas_topq_2019_23.pcl'):
    df = pd.read_pickle(f)
    if len(df) == 1:
        continue
    newDF = df.drop_duplicates()
    newDF.to_pickle(f)


In [12]:
pt.drop_duplicates()

,model,process,mST,mChi,mT,yDM,xsec (pb),MC Events,file,bin_355_381,...,bin_478_549,bin_478_549_Error,bin_549_633,bin_549_633_Error,bin_633_720,bin_633_720_Error,bin_720_836,bin_720_836_Error,bin_836_2000,bin_836_2000_Error
0,Top-FormFactorsOneLoop-UFO,$g g \to \bar{t} t$,300.0,5.0,172.5,1.0,-0.001036,75000,../../processFolders/gg2tt_formFactors_scan/Ev...,-0.000176,...,-0.000234,0.000008,-0.000145,0.000005,-0.000084,0.000003,-0.000058,0.000002,-0.000044,0.000001
